## Instalar librerias necesarias

In [1]:
pip install psycopg2-binary pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg2
import pandas as pd

# Paso 1: Crear base y tabla para CVEs del NIST
def create_database_and_table():
    try:
        conn = psycopg2.connect(
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432"
        )
        conn.autocommit = True
        cursor = conn.cursor()

        cursor.execute("SELECT datname FROM pg_database WHERE datname='nist_vulnerabilities';")
        if not cursor.fetchone():
            cursor.execute("CREATE DATABASE nist_vulnerabilities;")
            print(" Base de datos 'nist_vulnerabilities' creada correctamente.")
        else:
            print(" La base de datos 'nist_vulnerabilities' ya existe.")

        cursor.close()
        conn.close()

        # Conexión a la nueva base
        conn = psycopg2.connect(
            database="nist_vulnerabilities",
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432"
        )
        cursor = conn.cursor()

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS nist_cve_data (
                id SERIAL PRIMARY KEY,
                cve_id VARCHAR,
                published TIMESTAMP,
                last_modified TIMESTAMP,
                description TEXT,
                severity VARCHAR,
                cvss_score FLOAT,
                vector VARCHAR,
                exploitability_score FLOAT,
                impact_score FLOAT
            );
        """)
        print(" Tabla 'nist_cve_data' creada correctamente.")
        conn.commit()
        cursor.close()
        conn.close()

    except Exception as e:
        print(f" Error al crear la base o la tabla: {e}")

# Paso 2: Insertar 10 CVEs reales desde el CSV
def insert_real_cves():
    try:
        df = pd.read_csv("tratados.csv", parse_dates=["published", "last_modified"])
        df = df.dropna(subset=["cvss_score"])  # Asegurar registros válidos
        df_10 = df.head(10)

        conn = psycopg2.connect(
            database="nist_vulnerabilities",
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432"
        )
        cursor = conn.cursor()

        for _, row in df_10.iterrows():
            cursor.execute("""
                INSERT INTO nist_cve_data (
                    cve_id, published, last_modified, description,
                    severity, cvss_score, vector, exploitability_score, impact_score
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
            """, (
                row['cve_id'],
                row['published'],
                row['last_modified'],
                row['description'],
                row['severity'],
                row['cvss_score'],
                row['vector'],
                row['exploitability_score'],
                row['impact_score']
            ))

        conn.commit()
        print(" 10 vulnerabilidades insertadas correctamente.")
        cursor.close()
        conn.close()

    except Exception as e:
        print(f" Error al insertar CVEs: {e}")

# Paso 3: Mostrar registros insertados
def mostrar_registros():
    try:
        conn = psycopg2.connect(
            database="nist_vulnerabilities",
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432"
        )
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM nist_cve_data LIMIT 10;")
        rows = cursor.fetchall()

        print("\n Registros en la tabla:")
        for row in rows:
            print(row)

        cursor.close()
        conn.close()
    except Exception as e:
        print(f" Error al mostrar registros: {e}")

# Ejecución total
if __name__ == "__main__":
    create_database_and_table()
    insert_real_cves()
    mostrar_registros()


 Base de datos 'nist_vulnerabilities' creada correctamente.
 Tabla 'nist_cve_data' creada correctamente.
 10 vulnerabilidades insertadas correctamente.

 Registros en la tabla:
(1, 'CVE-1999-0241', datetime.datetime(1995, 11, 1, 5, 0), datetime.datetime(2024, 11, 20, 23, 28, 13, 143000), 'guessable magic cookies in x windows allows remote attackers to execute commands, e.g. through xterm.', 'high', 10.0, 'AV:N/AC:L/Au:N/C:C/I:C/A:C', 10.0, 10.0)
(2, 'CVE-1999-0179', datetime.datetime(1997, 1, 1, 5, 0), datetime.datetime(2024, 11, 20, 23, 28, 3, 757000), 'windows nt crashes or locks up when a samba client executes a "cd .." command on a file share.', 'medium', 5.0, 'AV:N/AC:L/Au:N/C:N/I:N/A:P', 10.0, 2.9)
(3, 'CVE-1999-0249', datetime.datetime(1997, 1, 1, 5, 0), datetime.datetime(2024, 11, 20, 23, 28, 14, 380000), 'windows nt rshsvc program allows remote users to execute arbitrary commands.', 'high', 7.2, 'AV:L/AC:L/Au:N/C:C/I:C/A:C', 3.9, 10.0)
(4, 'CVE-1999-0274', datetime.datetime(19